# Image Offsets: Process sequence

This notebook transforms the procedures developed in `Offsets_1.ipynb` into callable functions. 

These functions are used in a loop to process an entire sequence of images. Results are later examined in plots. 

The end product is a series of FITS tables, one per input image, that contain the star offsets in relation to the reference image. These tables should be used in a subsequent notebook to generate the actual arrays with pixel offsets that are used by drizzle to figure out the pixel mapping.

Originally this notebook was developed with the ISO 12800 data set as the test data. In the current version, it is configured to use a different data set. Statements specific to the original data set were comented out.

In [1]:
# %matplotlib widget

import time

import os, glob

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table
from astropy.stats import SigmaClip
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans

from photutils import centroids
from photutils.aperture import CircularAperture
from photutils.detection import DAOStarFinder
from photutils.background import Background2D, MedianBackground, ModeEstimatorBackground

import rawpy
import exifread

from datapath import DATA

## Initialization

Define values to be used in the processing functions, and throughout the script.

In [2]:
# ISO 12800 data set

# # images to be drizzled 
# data_dirpath = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/12800/light/')
# image_list = list(glob.glob(data_dirpath + '/*.ARW'))
# image_list.sort()

# # reference image - this will be the image which subsequent images will have their 
# # offsets computed against.
# ref_dirpath = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/12800/light/')
# reference_fname = os.path.join(ref_dirpath, 'DSC03770.ARW')

# # since the reference image was included in the image list by the glob command, it
# # has to be removed here
# image_list = image_list[1:]

In [3]:
# ISO 6400 data set

# # images to be drizzled 
# data_dirpath = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/6400/light/')
# image_list = list(glob.glob(data_dirpath + '/*.ARW'))
# image_list.sort()

# # reference image - this will be the image which subsequent images will have their 
# # offsets computed against. We want here that this be the same reference used for the
# # 12800 data set, so both data sets can be drizzled onto a common pixel grid.
# ref_dirpath = os.path.join(DATA,'astrophotography_data/MilkyWayPrettyBoy/12800/light/')
# reference_fname = os.path.join(ref_dirpath, 'DSC03770.ARW')

# # we need to process images in a given order: from the closest (in time) to the reference
# # image, to the most distant.
# image_list.reverse()

# image_list

In [4]:
# Andromeda data set

# images to be drizzled 
data_dirpath = os.path.join(DATA,'astrophotography_data/Andromeda_2022/135mm16s3200ISO')
image_list = list(glob.glob(data_dirpath + '/*.cutout.fits'))
image_list.sort()

# reference image - this will be the image which subsequent images will have their 
# offsets computed against. We want here that this be the same reference used for the
# 12800 data set, so both data sets can be drizzled onto a common pixel grid.
ref_dirpath = os.path.join(DATA,'astrophotography_data/Andromeda_2022/135mm16s3200ISO')
reference_fname_arw = os.path.join(ref_dirpath, 'DSC01263.ARW')
reference_fname = os.path.join(ref_dirpath, 'DSC01263.cutout.fits')

# since the reference image was included in the image list by the glob command, it
# has to be removed here
image_list = image_list[1:]

print(len(image_list))
image_list

899


['/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01263.cutout.fits',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01264.cutout.fits',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01265.cutout.fits',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01266.cutout.fits',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01267.cutout.fits',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01268.cutout.fits',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01269.cutout.fits',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01270.cutout.fits',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01271.cutout.fits',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01272.cutout.fits',


In [5]:
# normalization factors to get the roundest star images (as per Offsets_1 notebook)
red_norm = 1.6  
blue_norm = 1.39

# parameters to control background subtraction
bkg_cell_footprint = (100, 100)
bkg_filter = (11, 11)

bkg_sigma_clip = SigmaClip(sigma=5.)
bkg_kernel = Gaussian2DKernel(x_stddev=1)
bkg_estimator = ModeEstimatorBackground()

# parameters for star finding
# dao_fwhm = 5.   # ISO 12800
# dao_fwhm = 6.0    # ISO 6400
dao_fwhm = 7.0    # Andromeda
# dao_threshold = 2000.   # ISO 12800
# dao_threshold = 1200.   # ISO 6400
# dao_threshold = 600.   # Andromeda 6400
dao_threshold = 200.   # Andromeda 3200
sharplo = 0.1
sharphi = 1.0
roundlo = -0.9 
roundhi = 0.9

# proximity = 3.5   # ISO 12800
# proximity = 4.   # ISO 6400
proximity = 10.   # Andromeda

# tolerance to detect condition that search is stuck
tol = 0.5

In [6]:
# read reference image - we need to read it here to get the camera color array specification.
raw = rawpy.imread(reference_fname_arw)
ref_imarray = raw.raw_image_visible.astype(float)

In [7]:
# masks that isolate the RGB pixels - these are camera-dependent and work with all images
colors_array = raw.raw_colors_visible

red_mask = np.where(colors_array == 0, 1, 0)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = green_mask_1 | green_mask_2

blue_mask = np.where(colors_array == 2, 1, 0)

Using normalizations derived from passband spectral response is *much* better than using normalizations derived from minimization of sky background variance (contrary to the initial finding in notebook Offsets_1). The likely cause is that the spectral-based normalization creates more well-behaved star images. The high variance in sky background doesn't seem to get in the way of detecting stars.

The best run with smooth background on the ISO 12800 data generated 240 detections with a complete data set. The same run but with spectral-based color band normalizations resulted in 550 detections.

## Processing functions

In [8]:
# computes position offsets between two tables. 
def get_offsets(sources, sources_prev):

    sources.add_column(np.nan, name='xoffset')
    sources.add_column(np.nan, name='yoffset')
    sources.add_column(0.0, name='xoffset_prev')
    sources.add_column(0.0, name='yoffset_prev')
    sources.add_column(0, name='ref_row')
    sources.add_column(0, name='prev_row')

    # loop over rows in previous table
    for row_index_prev in range(len(sources_prev)):
        # index in reference table
        ref_row = sources_prev[row_index_prev]['ref_row']

        # if previous table does not contain a pointer to 
        # the reference table, ignore.
        if ref_row == 0:
            continue

        # get position in previous table
        x_prev = sources_prev[row_index_prev]['xcentroid']
        y_prev = sources_prev[row_index_prev]['ycentroid']

        # loop over rows in current table
        for row_index in range(len(sources)):
            x = sources[row_index]['xcentroid']
            y = sources[row_index]['ycentroid']

            # offsets in relation to previous table - these are the ones to check for proximity
            x_off_previous = x - x_prev
            y_off_previous = y - y_prev

            # check for proximity, and store relevant info if found
            if abs(x_off_previous) <= proximity and abs(y_off_previous) <= proximity:

                # offsets in relation to reference table
                sources[row_index]['xoffset'] = x - sources_ref[ref_row]['xcentroid']
                sources[row_index]['yoffset'] = y - sources_ref[ref_row]['ycentroid']

                # offsets in relation to previous table
                sources[row_index]['xoffset_prev'] = x_off_previous
                sources[row_index]['yoffset_prev'] = y_off_previous

                # store pointers to rows in reference and previous tables
                sources[row_index]['ref_row'] = ref_row
                sources[row_index]['prev_row'] = row_index_prev
                
                #TODO 
                # instead of breaking, do an estimate of where the centroid would be,
                # given the current position, and the offsets from the previous table.
                # In other words, repeat the offset from the previous table. See if this
                # will cause the finding algorithm to pick up in the next image.

                break # if there is another star that matches the criterion, just ignore it
                
    return sources

In [9]:
def subtract_background(imarray, red_norm=1.0, blue_norm=1.0):

    # red_norm and blue_norm are normalization parameters applied to the R and B bands (assume
    # G=1) in order to make the star images as well-behaved as possible, in terms of being 
    # well represented, on average, by the daofind Gaussian. Ideally a different normalization 
    # should be applied to each star, depending on its color index, but this will be left as
    # a possible (but not very likely) future improvement. For now, we assume that an average,
    # frame-wide single normalization should suffice (statistically).
    
    # separate color bands
    red_array = imarray * red_mask
    green_array = imarray * green_mask
    blue_array = imarray * blue_mask
    
    # interpolate over the masked pixels in each band, so the background estimator 
    # is presented with a smooth array entirely filled with valid data
    red_array[red_array == 0.0] = np.nan
    green_array[green_array == 0.0] = np.nan
    blue_array[blue_array == 0.0] = np.nan

    red_array = interpolate_replace_nans(red_array, bkg_kernel)
    green_array = interpolate_replace_nans(green_array, bkg_kernel)
    blue_array = interpolate_replace_nans(blue_array, bkg_kernel)

    red_array[np.isnan(red_array)] = 0.
    green_array[np.isnan(green_array)] = 0.
    blue_array[np.isnan(blue_array)] = 0.
    
    # fit background model to each smoothed-out color band
    red_bkg = Background2D(red_array, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)
    green_bkg = Background2D(green_array, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)
    blue_bkg = Background2D(blue_array, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=bkg_sigma_clip, bkg_estimator=bkg_estimator)

    # subtract background from each masked color array
    subtracted = imarray - red_bkg.background * red_mask - \
                           green_bkg.background * green_mask - \
                           blue_bkg.background * blue_mask

    # after background subtraction, apply color band normalization. This has to be done separately
    # from step above for the background on each band to remain zero on average.
    subtracted = (subtracted * red_mask * red_norm) + \
                 (subtracted * green_mask) + \
                 (subtracted * blue_mask * blue_norm)

    return subtracted

In [10]:
class CentroidFinder():
    def __init__(self, x_estimate, y_estimate, size, sky, threshold):
        self.x_estimate = x_estimate
        self.y_estimate = y_estimate
        self.sky = sky
        self.threshold = threshold
        self.size = int(size)
        
        self.result = Table()
        
    def find(self, image):
#         try:
        x, y = centroids.centroid_sources(image, self.x_estimate, self.y_estimate, 
                                          box_size=self.size, centroid_func=centroids.centroid_com) 
        self.result['xcentroid'] = x
        self.result['ycentroid'] = y
#         except ValueError as e:
#             raise ValueError(str(e))

        return self.result                

In [11]:
# creates a table with star positions, given a path to an image file
def find_stars(path, sources_prev=None):


    f = fits.open(path)
    imarray = f[1].data
    f.close()    
    
    
# TODO background was already subtracted.
#         subtracted = subtract_background(imarray, red_norm=red_norm, blue_norm=blue_norm)
    subtracted = imarray

    global_median = np.median(subtracted)

    # find stars
    if sources_prev is None:
        daofind = DAOStarFinder(fwhm=dao_fwhm, sky=global_median, threshold=dao_threshold,
                                sharplo=sharplo, sharphi=sharphi,
                                roundlo=roundlo, roundhi=roundhi) 
        sources = daofind(subtracted)
    else:
        # offsets are added in reverse, to generate an estimate further away from the current position.
        x_estimate = sources_prev['xcentroid'] - sources_prev['xoffset_prev'] 
        y_estimate = sources_prev['ycentroid'] - sources_prev['yoffset_prev'] 

#             positions = [(x,y) for x,y in zip(x_estimate, y_estimate)]
#             daofind = DAOStarFinder(xycoords=np.array(positions), fwhm=dao_fwhm, 
#                                     sky=global_median,
#                                     threshold=dao_threshold,
#                                     sharplo=sharplo, sharphi=sharphi,
#                                     roundlo=roundlo, roundhi=roundhi) 
#             sources = daofind(subtracted)

        try:
            finder = CentroidFinder(x_estimate, y_estimate, dao_fwhm*2+1, global_median, dao_threshold)
            sources = finder.find(subtracted)  # 1
        except ValueError:
            print("Found points outside image. Discard them.")
            row = 0
            for xe, ye in zip(x_estimate, y_estimate):
                if xe < 10 or xe >= imarray.shape[1]-10 or ye < 10 or ye >= imarray.shape[0]-10:
                    print(xe, ye, imarray.shape[1], imarray.shape[0])
                    x_estimate[row] = np.NaN
                    y_estimate[row] = np.NaN
                    print(xe, ye, imarray.shape[1], imarray.shape[0])
                row += 1

            sources_reject = Table({'xcentroid': x_estimate, 'ycentroid': y_estimate})

            print("cleaning nans...")
            
            sources_clean = clean_nans(sources_reject, colname='xcentroid') 

            x_estimate = sources_clean['xcentroid'] 
            y_estimate = sources_clean['ycentroid']

            try:
                finder = CentroidFinder(x_estimate, y_estimate, dao_fwhm*2+1, global_median, dao_threshold)
                sources = finder.find(subtracted)  # 2
            except ValueError:
                print("trying again...")
                row = 0
                for xe, ye in zip(x_estimate, y_estimate):
                    print("in loop 2 ", row, xe, ye)
                    if xe < 1 or xe >= imarray.shape[1]-1 or ye < 1 or ye >= imarray.shape[0]-1:
                        print(xe, ye, imarray.shape[1], imarray.shape[0])
                        sources['xcentroid'][row] = np.NaN
                        sources['ycentroid'][row] = np.NaN
                        print(xe, ye, imarray.shape[1], imarray.shape[0])
                    row += 1

                sources = clean_nans(sources) 
                print("cleaning nans...")

                x_estimate = sources['xcentroid'] 
                y_estimate = sources['ycentroid']
                try:
                    finder = CentroidFinder(x_estimate, y_estimate, dao_fwhm*2+1, global_median, dao_threshold)
                    sources = finder.find(subtracted)  # 3
                except ValueError:
                    print("got error again...")
                    for xe,ye in zip(x_estimate, y_estimate):
                        if xe < 1 or xe >= imarray.shape[1]-1 or ye < 1 or ye >= imarray.shape[0]-1:
                            print(xe, ye)
                    # give up on image
                    return None, subtracted, global_median
            
    return sources, subtracted, global_median

In [12]:
# keep only the NaN-free entries
def clean_nans(sources, colname='xoffset'):
    has_nan = np.zeros(len(sources), dtype=bool)
    xoff = np.array(sources[colname])
    has_nan |= np.isnan(xoff)
    return sources[~has_nan]

## Process sequence

In [13]:
# find stars in reference image
sources_ref, subtracted_ref, global_median = find_stars(reference_fname)

# positions storage
positions_tables = {}
positions_tables[reference_fname] = sources_ref

# array for stacking subtracted images
image_stack = np.zeros_like(subtracted_ref)

# add default offset columns to reference table
sources_ref.add_column(0., name='xoffset')
sources_ref.add_column(0., name='yoffset')
sources_ref.add_column(0., name='xoffset_prev')
sources_ref.add_column(0., name='yoffset_prev')

# in ref table, rows point to themselves
sources_ref.add_column(sources_ref['id']-1, name='ref_row')
sources_ref.add_column(sources_ref['id']-1, name='prev_row')

# force reference image to be the "previous" image
sources_prev = sources_ref

sources_ref

id,xcentroid,ycentroid,sharpness,roundness1,roundness2,npix,sky,peak,flux,mag,xoffset,yoffset,xoffset_prev,yoffset_prev,ref_row,prev_row
int64,float64,float64,float64,float32,float64,int64,float32,float32,float64,float64,float64,float64,float64,float64,int64,int64
1,1610.1836851626579,8.32357070113292,0.8188604294706963,0.09169102,-0.07167069476525272,81,1.2459884,194.09084,-99.42093002796173,nan,0.0,0.0,0.0,0.0,0,0
2,1490.7338065344202,10.145649482004679,0.5947028392411983,0.08727369,0.14531953084174104,81,1.2459884,149.98093,-99.37095880508423,nan,0.0,0.0,0.0,0.0,1,1
3,2371.571641188357,11.631308630270116,0.4151808104281007,-0.11167616,-0.31471798818829455,81,1.2459884,102.59549,-99.39543771743774,nan,0.0,0.0,0.0,0.0,2,2
4,81.80196574863612,25.890440805151112,0.8209332244106547,-0.11070844,0.02100754669568776,81,1.2459884,187.79784,-99.47393524646759,nan,0.0,0.0,0.0,0.0,3,3
5,485.06340274887805,40.86253840991864,0.29596526798798645,-0.20446508,-0.14945570976911232,81,1.2459884,63.898483,-99.76136076450348,nan,0.0,0.0,0.0,0.0,4,4
6,837.5456632178918,45.90419579198981,0.8140061056171397,-0.04905482,-0.06304656737300543,81,1.2459884,883.84357,-93.8477520942688,nan,0.0,0.0,0.0,0.0,5,5
7,475.4812279338103,50.113261782666775,0.5741948421548884,-0.5488586,-0.24248690474334533,81,1.2459884,99.22833,-99.83654749393463,nan,0.0,0.0,0.0,0.0,6,6
8,1621.2137672864417,52.354895950363655,0.6621124197469024,-0.042910315,0.030759297922069617,81,1.2459884,108.8975,-99.90986382961273,nan,0.0,0.0,0.0,0.0,7,7
9,782.4284701284599,62.209558569118336,0.9729837411837087,0.35384363,-0.0792860868803069,81,1.2459884,165.00467,-99.79477775096893,nan,0.0,0.0,0.0,0.0,8,8


In [14]:
sources_prev = sources_ref

# loop over list of images to be drizzled
for file_path in image_list:
    
    # find stars
#     t1 = time.time()
    sources, subtracted, global_median = find_stars(file_path, sources_prev=sources_prev)
#     t2 = time.time()
#     print("find stars: " , (t2-t1), " sec.")
    
    # compute offsets
#     t1 = time.time()
    sources_current = get_offsets(sources, sources_prev)
#     t2 = time.time()
#     print("get offsets: " , (t2-t1), " sec.")
    
    sources_current_no_nan = clean_nans(sources_current)
    
    positions_tables[file_path] = sources_current_no_nan
    
    # check that sources_current_no_nan has different offsets from sources_prev. If no
    # differences, break out of loop.
    if len(sources_current_no_nan) == len(sources_prev):
        xoff_c = np.array(sources_current_no_nan['xoffset'])
        yoff_c = np.array(sources_current_no_nan['yoffset'])
        xoff_p = np.array(sources_prev['xoffset'])
        yoff_p = np.array(sources_prev['yoffset'])
        if np.all(np.isclose(xoff_c, xoff_p, atol=tol)) and np.all(np.isclose(yoff_c, yoff_p, atol=tol)):
            print("Identical solution: search is stuck")
            break
    
    # for next iteration, current table becomes previous
    sources_prev = sources_current_no_nan
    
    # update image stack
    image_stack += subtracted
    
    print(file_path, len(sources_current_no_nan), global_median)

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01263.cutout.fits 472 1.2459884
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01264.cutout.fits 459 1.2685542
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01265.cutout.fits 444 1.2923403
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01266.cutout.fits 435 1.2358317
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01267.cutout.fits 430 1.2624539
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01268.cutout.fits 428 1.28564
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01269.cutout.fits 427 1.3338683
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01270.cutout.fits 426 1.3079522
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01271.cutout.fits 425 1.302424
/Users/busko/

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01338.cutout.fits 411 2.0366316
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01339.cutout.fits 411 2.0586188
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01340.cutout.fits 411 2.073257
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01341.cutout.fits 411 2.0898292
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01342.cutout.fits 410 2.0315938
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01343.cutout.fits 410 2.1304216
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01344.cutout.fits 409 2.097295
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01345.cutout.fits 409 2.129389
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01346.cutout.fits 409 2.1669595
/Users/busko/

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01412.cutout.fits 408 2.7851155
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01413.cutout.fits 408 2.7459512
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01414.cutout.fits 408 2.7518234
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01415.cutout.fits 408 2.7750149
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01416.cutout.fits 408 2.8098397
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01417.cutout.fits 408 2.7963095
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01418.cutout.fits 408 2.7562623
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01419.cutout.fits 408 2.8225152
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01420.cutout.fits 408 2.7494893
/Users/bus

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01487.cutout.fits 406 3.1688485
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01488.cutout.fits 406 3.1784525
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01489.cutout.fits 406 3.1689591
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01490.cutout.fits 406 3.1721697
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01491.cutout.fits 406 3.2158523
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01492.cutout.fits 406 3.201133
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01493.cutout.fits 406 3.2032106
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01494.cutout.fits 406 3.218004
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01495.cutout.fits 406 3.1949677
/Users/busko

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01562.cutout.fits 404 3.329145
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01563.cutout.fits 404 3.3789043
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01564.cutout.fits 404 3.356875
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01565.cutout.fits 404 3.361744
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01566.cutout.fits 404 3.3737094
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01567.cutout.fits 404 3.4083776
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01568.cutout.fits 404 3.3578238
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01569.cutout.fits 404 3.362996
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01570.cutout.fits 404 3.3878255
/Users/busko/P

...

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01859.cutout.fits 387 3.2824154
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01860.cutout.fits 387 3.2215693
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01861.cutout.fits 387 3.2346814
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01862.cutout.fits 387 3.248142
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01863.cutout.fits 387 3.199544
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01864.cutout.fits 386 3.2126021
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01865.cutout.fits 386 3.2296305
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01866.cutout.fits 386 3.180059
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01867.cutout.fits 386 3.2138271
/Users/busko/

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01934.cutout.fits 383 2.9152935
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01935.cutout.fits 383 2.9014473
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01936.cutout.fits 383 2.9080129
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01937.cutout.fits 383 2.9455686
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01938.cutout.fits 383 2.913505
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01939.cutout.fits 383 2.927308
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01940.cutout.fits 383 2.9641871
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01941.cutout.fits 383 2.9051192
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01942.cutout.fits 383 2.9272742
/Users/busko

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02009.cutout.fits 370 2.3516345
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02010.cutout.fits 370 2.34525
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02011.cutout.fits 370 2.3147073
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02012.cutout.fits 370 2.33106
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02013.cutout.fits 370 2.2779284
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02014.cutout.fits 370 2.3225863
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02015.cutout.fits 370 2.274033
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02016.cutout.fits 370 2.2687259
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02017.cutout.fits 370 2.2647386
/Users/busko/Pr

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02084.cutout.fits 358 1.5612234
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02085.cutout.fits 358 1.5650016
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02086.cutout.fits 358 1.5416985
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02087.cutout.fits 356 1.4528693
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02088.cutout.fits 355 1.4599819
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02089.cutout.fits 355 1.4760525
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02090.cutout.fits 355 1.4929175
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02091.cutout.fits 355 1.4372966
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02092.cutout.fits 355 1.4809554
/Users/bus

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02157.cutout.fits 335 0.6813534
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02158.cutout.fits 335 0.7225381
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02159.cutout.fits 335 0.66223943
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02160.cutout.fits 335 0.66480076
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02161.cutout.fits 335 0.6736109


In [15]:
sources_current_no_nan

xcentroid,ycentroid,xoffset,yoffset,xoffset_prev,yoffset_prev,ref_row,prev_row
float64,float64,float64,float64,float64,float64,int64,int64
892.6645682904102,210.00412490085915,55.11890507251837,164.09992910886933,0.10474920913816277,-0.8002918740402833,5,0
1628.591511930826,173.8053140131165,61.20362887165311,108.81118104648753,-0.10146010371545344,-1.290756715741992,9,1
1809.9205725609659,173.32457635483598,55.983742813328945,95.1903331892859,0.062473547978925126,-0.6257095282558112,11,2
1767.6394080279547,180.3777861617249,57.26907970082061,98.58198189780606,-0.1730134390782041,-0.7194577705010943,13,3
981.7071936685724,242.44609217208819,60.54023422081286,157.389234112265,0.25278537226665776,-1.8893681669273974,15,4
2287.6746530713344,201.6647424697541,51.51110875997938,57.58373197671196,0.18873489795305431,-1.9077358268976639,23,5
1748.0788329578681,254.54152871879867,62.177040286231886,98.68289278807487,0.630509872130915,-1.5051777725064142,24,6
1559.3730400546626,275.6295015624948,68.22315255233639,114.85434180493041,-0.8553336161160132,-0.40031104851908594,25,7
1352.9475944980948,301.20913726854724,69.06911278843813,132.86952183159335,-1.7195831186004398,1.344575485761709,26,8


In [16]:
# positions = [(x,y) for x,y in zip(sources_current_no_nan['xcentroid'], sources_current_no_nan['ycentroid'])]
# positions_ref = [(x,y) for x,y in zip(sources_ref['xcentroid'], sources_ref['ycentroid'])]

# apertures = CircularAperture(positions, r=5.)
# apertures_ref = CircularAperture(positions_ref, r=5.)

# plt.figure(figsize=[9, 6])
# plt.imshow(image_stack, vmin=-10, vmax=10000, cmap='binary')
# plt.colorbar()
# _ = apertures.plot(color='red')
# _ = apertures_ref.plot(color='yellow')

In [17]:
yr0 = 0
yr1 = image_stack.shape[1]
xr0 = 0
xr1 = image_stack.shape[0]
# yr0 = 1000
# yr1 = 1200
# xr0 = 500
# xr1 = 700

aslice = np.index_exp[yr0:yr1,xr0:xr1]

In [18]:
# plt.figure(figsize=[9, 6])
# plt.imshow(image_stack[aslice], vmin=-10, vmax=30000, cmap='binary')
# plt.colorbar()

# for file_path in list(positions_tables.keys()):
#     positions_t = positions_tables[file_path]
#     positions = [(x-xr0,y-yr0) for x,y in zip(positions_t['xcentroid'], positions_t['ycentroid'])]
#     apertures = CircularAperture(positions, r=0.1)
#     _ = apertures.plot(color='red')

## Plot only complete sequence stars

Stars with a complete sequence of measured centroids are the ones that make to the last processed image/table in the sequence. Thus, starting from the end image and going backwards (in processing order), we ensure we pick only the complete sequence stars.

In [19]:
# # start from last processed table. 
# file_path_last = list(positions_tables.keys())[-1]
# positions_last = positions_tables[file_path_last]
# refrow_last = positions_last['ref_row']

# file_path_ref = list(positions_tables.keys())[0]  # ref table
# positions_ref = positions_tables[file_path_ref]

# yr0 = 0
# yr1 = image_stack.shape[1]
# xr0 = 0
# xr1 = image_stack.shape[0]
# # yr0 = 1000
# # yr1 = 1200
# # xr0 = 500
# # xr1 = 700

# aslice = np.index_exp[yr0:yr1,xr0:xr1]

# plt.figure(figsize=[9, 6])
# plt.imshow(image_stack[aslice], vmin=-10, vmax=30000, cmap='binary')
# plt.colorbar()

# for i, row in enumerate(refrow_last):
#     ref_row = positions_ref[row]
    
#     x0 = ref_row['xcentroid']
#     y0 = ref_row['ycentroid']
#     x1 = positions_last['xcentroid'][i]
#     y1 = positions_last['ycentroid'][i]

# #     plt.plot( [x0,x1], [y0,y1], 'r', linewidth=1, markersize=1)

#     if x0>xr0 and x0<xr1 and y0>yr0 and y0<yr1 and x1>xr0 and x1<xr1 and y1>yr0 and y1<yr1:
    
#         plt.plot( [x0-xr0,x1-xr0], [y0-yr0,y1-yr0], 'r', linewidth=1, markersize=1)

## Write tables

In [20]:
from astropy.io import fits

keys = list(positions_tables.keys())

for key in keys:
    dirname = os.path.dirname(key)
    fname = os.path.basename(key)
    imagename = fname.split('.')[0]
    tablename = os.path.join(dirname, imagename + '.offsets_table.fits')
    
    table = positions_tables[key]
    
    table.write(tablename, overwrite=True)

    print(tablename)

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01263.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01264.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01265.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01266.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01267.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01268.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01269.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01270.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01271.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01378.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01379.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01380.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01381.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01382.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01383.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01384.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01385.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01386.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01464.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01465.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01466.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01467.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01468.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01469.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01470.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01471.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01472.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01548.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01549.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01550.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01551.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01552.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01553.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01554.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01555.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01556.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01633.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01634.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01635.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01636.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01637.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01638.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01639.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01640.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01641.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS

...

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01801.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01802.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01803.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01804.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01805.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01806.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01807.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01808.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01809.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01886.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01887.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01888.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01889.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01890.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01891.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01892.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01893.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01894.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01974.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01975.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01976.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01977.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01978.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01979.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01980.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01981.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC01982.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02057.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02058.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02059.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02060.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02061.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02062.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02063.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02064.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02065.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS

/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02146.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02147.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02148.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02149.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02150.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02151.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02152.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02153.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200ISO/DSC02154.offsets_table.fits
/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm16s3200IS